# Categories and traits

## Motivation

One of the small utilities in the standard library is the `advance` function, which moves an iterator with a given number of steps :

In [1]:
template<typename IterT, typename DistT>   // move iter d units forward
void advance( IterT & iter, DistT d ) ;    // or backward (if d<0)

Conceptually, it's just a matter of doing `iter+=d`, but the `+=` operator is only available for direct access iterators. For the others, `++` or `--` opeartos must be applied as many times as necessary.

`advance()` can be written based on the smallest common iterator behavior, but it would be a shame to have linear time execution all the time, whereas for direct access iterators we could benefit from a constant time execution. A pseudo-code can be written like this :

In [2]:
template<typename IterT, typename DistT>
void advance(IterT& iter, DistT d)
 {
  if (iter is a random access iterator)
   {
     iter += d ;                                      // use iterator arithmetic
   }                                                  // for random access iters
  else
   {
    if (d >= 0) { while (d--) ++iter ; }              // use iterative calls to
    else { while (d++) --iter ; }                     // ++ or -- for other
   }                                                  // iterator categories
 }

input_line_8:4:7: error: unknown type name 'iter'
  if (iter is a random access iterator)
      ^
input_line_8:4:12: error: variable declaration in condition must have an initializer
  if (iter is a random access iterator)
           ^


Interpreter Error: 

How to write code that selects the best implementation at the compilation, depending on whether `+=` is available or not for `IterT`?

## Iterators and categories for the Standard library

There are five categories of iterators, offering increasing functionality :
* **Input iterators** can only go forward, one step at a time, can only read what they point to,
 and can only read it once. They are designed like the play pointer of a file.
 `istream_iterator` is representative of this category.
* **Output iterators** are analogous to the fact that they can only write what they point to.
 `ostream_iterator` is representative of this category.
 `istream_iterator` and `ostream_iterator` are very limited and are only suitable for single pass algorithms.
* The **front iterators** combine the capacities of the two previous ones, and can read/write the data pointed several times.
  Therefore, they are  suitable for multiple pass algorithms.
  Simply linked list `forward_list` and hash-table containers such as `unordered_set` provide forward iterators.
* **Bidirectional iterators** add the ability to reverse.
 Iterators of Standard containers can be find in this category
  such as `list`, `set`, `multiset`, `map`, and `multimap`.
* **Direct access iterators** add the ability to jump forward and backward,
  in constant time, comparable to ordinary pointer arithmetic.
  Leave the iterators of the `vector`, `deque`, and `string` classes to this category.

For each of these categories, the standard library defines an empty structure :

```c++
struct input_iterator_tag {} ;
struct output_iterator_tag {} ;
struct forward_iterator_tag: public input_iterator_tag {} ;
struct bidirectional_iterator_tag: public forward_iterator_tag {} ;
struct random_access_iterator_tag: public bidirectional_iterator_tag {} ;
```

The above inheritance expresses a "is kind of" relation, and it will be seen later how to use it.

To mark their belonging in one of the above categories, most iterators in the standard library contain a nested type alias named `iterator_category`, which points to the corresponding above empty structure.

## Using these categories

One can imagine to write `advance` like this :

In [3]:
template<typename IterT, typename DistT>
void advance( IterT & iter, DistT d )
 {
  if (typeid(typename IterT::iterator_category) ==
      typeid(std::random_access_iterator_tag))
   { iter += d ; } 
  else
   {
    if (d >= 0) { while (d--) ++iter ; }
    else { while (d++) --iter ; }
   }
 }

This looks promising, but it is not satisfactory to keep an `if`, which will consume time at execution and make the code heavier, while all the information is available at the compilation, and should make it possible to know which branch of the `if` will always be applied. In fact, this code does not even compile, because in the presence of a non-direct iterator, the compiler tries to compile the whole function, and fails on the `+=`.

**We look for a way to choose, at compile time, which piece of code should be used, depending on the involved types**. This mean has already existed in C++ : it is the *resolution of the overload*. Indeed, the overload makes it possible to have several functions with the same name, and when a call to this function name must be compiled, the compiler chooses, according to the arguments of the call, the real function to be executed. One just has to insert an instance of one of the above iterator categories, and it will trigger the selection of the most appropriate function.

To apply this principle to `advance()`, most of its content must be delegated to an external parameterized function, in which an additional category type argument must be included :

In [4]:
#include <iostream>

In [5]:
template< typename IterT, typename DistT >              // use this impl for
void do_advance( IterT & iter, DistT d,                  // random access
                std::random_access_iterator_tag )       // iterators
 {
  iter += d ;
  std::cout<<"(random advance)"<<std::endl ;
 }

In [6]:
template< typename IterT, typename DistT >              // use this impl for
void do_advance( IterT & iter, DistT d,                  // bidirectional
                std::bidirectional_iterator_tag )        // iterators
 {
  if (d >= 0) { while (d--) ++iter ; }
  else { while (d++) --iter ; }
  std::cout<<"(bidirectional advance)"<<std::endl ;
 }

In [7]:
template< typename IterT, typename DistT >              // use this impl for
void do_advance( IterT & iter, DistT d,                  // input iterators
                std::input_iterator_tag )
 {
  if (d < 0 )
   { throw std::out_of_range("Negative distance") ; } // see below
  while (d--) ++iter ;
  std::cout<<"(forward advance)"<<std::endl ;
 }

Because the `forward_iterator_tag` category inherits from `input_iterator_tag`, the corresponding `doAdvance()` can handle both cases, and a specific implementation for each type of input iterator is now obtained. All that's left to do in `advance()` is to call `doAdvance()` dragging the appropriate temporary object :

In [8]:
template<typename IterT, typename DistT>
void my_advance( IterT & iter, DistT d )
 {
  do_advance
   (
    iter, d,                            // call the version of doAdvance
    typename IterT::iterator_category() // appropriate for iter's iterator category
   ) ;                                  
 }                                  

In [18]:
#include <forward_list>
#include <list>
#include <vector>

In [19]:
std::forward_list<int> fl_data = { 1, 2, 3, 4, 5 } ;
auto fl_itr = fl_data.begin() ;
my_advance(fl_itr,2) ;

(forward advance)


In [20]:
std::list<int> l_data = { 1, 2, 3, 4, 5 } ;
auto l_itr = l_data.begin() ;
my_advance(l_itr,2) ;

(bidirectional advance)


In [21]:
 
std::vector<int> v_data = { 1, 2, 3, 4, 5 } ;
auto v_itr = v_data.begin() ;
my_advance(v_itr,2) ;    

(random advance)


Categories usage can be summarized as the following :
* create a set of work functions (`do_advance` in the example), parameterized or not,
  which differ by an argument of type category, and optimize each function according to this category.
* create a main function (`my_advance` in the example), parameterized or not,
  which calls the working function by adding the category information.

## Traits usage

The previous approach still suffers from a flaw : because a nested type must be defined in the iterators, the raw C++ pointers cannot be processed in this way, although they are also iterators.

In [10]:
int data[5] = { 1, 2, 3, 4, 5 } ;
auto itr = &data[0] ;
my_advance(itr,2) ;

input_line_14:7:14: error: type 'int *' cannot be used prior to '::' because it has no members
    typename IterT::iterator_category() // appropriate for iter's iterator category
             ^
input_line_17:4:1: note: in instantiation of function template specialization '__cling_N58::my_advance<int *, int>' requested here
my_advance(itr,2) ;
^


Interpreter Error: 

Whatever set of classes you want to "tag" with categories, you will get a similar problem if some of those classes are not under your control, and cannot be modified. They have to be extended **from outside**.

A solution is to create a template, parameterized by the type to extend, as well as a set of specializations. For the iterators of the standard library, this template exists and it is called `iterator_traits` :

In [11]:
template<typename IterT>  // template for information about
struct iterator_traits ;  // iterator types

By convention, it is a `struct` (elements are public by default), whose name includes "traits", and which contains only elements accessible statically through the class operator, such as typedefs or static variables. In the case of `iterator_traits`, a `typedef` is here : it is named `iterator_category` and refers to one of the usual categories.

Writing `iterator_traits` relies on two cases. In one hand, the iterators of the containers in the standard library are already supposed to define internally their own alias `iterator_category`, which can be taken from the general default implementation :

In [12]:
template<typename IterT>
struct iterator_traits
 {
  typedef typename IterT::iterator_category iterator_category ;
  // ...
 } ;

It is then sufficient to complete with a partial specialization to deal with the case of ordinary pointers, which can be considered as belonging to the category "random access" :

In [14]:
template< typename IterT >               // partial template specialization
struct iterator_traits< IterT * >      // for built-in pointer types
 {
  typedef std::random_access_iterator_tag iterator_category ;
  // ...
 } ;

The implementation of `my_advance()` can be revised using traits, which make it usable with simple pointers as well :

In [15]:
template<typename IterT, typename DistT>
void my_advance_2( IterT & iter, DistT d )
 {
  do_advance
   (    
    iter, d,                                                  // call the version of doAdvance
    typename std::iterator_traits<IterT>::iterator_category() // appropriate for iter's iterator category
   ) ;                                  
 }                                  

In [16]:
int data[5] = { 1, 2, 3, 4, 5 } ;
auto itr = &data[0] ;
my_advance_2(itr,2) ;

(random advance)


## Traits in the standard library

Traits are widely used in the standard library. There is `iterator_traits`, of course, which defines `iterator_category` together with adiitional information, including the very useful `value_type`, the type of the pointed data.

Traits can also define functions, as does `numeric_limits`, which provides information about numeric types, such as their minimum and maximum values.

In [24]:
#include<iostream>
#include <iterator>
#include <limits>

In [25]:
template< typename Itr >
typename std::iterator_traits<Itr>::value_type max ( Itr begin, Itr end )
 {
  using value_type = typename std::iterator_traits<Itr>::value_type ;
  value_type max_value = std::numeric_limits<value_type>::min() ;
  while ( begin != end )
   {
    if (*begin>max_value) max_value = *begin ;
    begin++ ;
   }
  return max_value ;
 }

In [26]:
int data[5] = { 1, 2, 3, 4, 5 } ;
std::cout<<max(&data[0],&data[5])<<std::endl ;

5


Traits can also define static variables. Within the `type_traits` header file, the library defines a very large number of traits that allow you to test the properties of the type `T`, passed as a parameter. Rather than defining a single large structure, which would contain a boolean variable for each property, the authors have preferred to make a distinct trait for each property, with a single boolean variable named `value`. This approach improves the "composability" of those traits.

Below, a parameterized function has been defined to ensure that it is fast, which requires the type `T` to be "movable". The use of a `static_assert`, and of the traits `is_move_constructible` and `is_move_assignable` allows to ensure it :

In [28]:
#include <iostream>
#include <string>
#include <type_traits>

In [29]:
template< typename T >
void swap( T & v1, T & v2 )
 {
  static_assert(std::is_move_constructible<T>::value,"Swap requires moving");
  static_assert(std::is_move_assignable<T>::value,"Swap requires moving");
  auto tmp = std::move(v1) ;
  v1 = std::move(v2) ;
  v2 = std::move(tmp) ;
 }

In [30]:
std::string mot1 = "world" ;
std::string mot2 = "hello" ;
swap(mot1,mot2) ;
std::cout<<mot1<<" "<<mot2<<std::endl ;

hello world


In [31]:
char mot1[5] = { 'w','o','r','l','d' } ;
char mot2[5] = { 'h','e','l','l','o' } ;
swap(mot1,mot2) ;
for ( auto c : mot1 ) std::cout<<c ;
std::cout<<" " ;
for ( auto c : mot2 ) std::cout<<c ;
std::cout<<std::endl ;

input_line_36:4:3: error: static_assert failed "Swap requires moving"
  static_assert(std::is_move_constructible<T>::value,"Swap requires moving");
  ^             ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
input_line_38:4:1: note: in instantiation of function template specialization '__cling_N522::swap<char [5]>' requested here
swap(mot1,mot2) ;
^
input_line_36:5:3: error: static_assert failed "Swap requires moving"
  static_assert(std::is_move_assignable<T>::value,"Swap requires moving");
  ^             ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
input_line_36:7:6: error: array type 'char [5]' is not assignable
  v1 = std::move(v2) ;
  ~~ ^
input_line_36:8:6: error: array type 'char [5]' is not assignable
  v2 = std::move(tmp) ;
  ~~ ^


Interpreter Error: 

## To remember

- In order to organize a set of classes, some empty types can be defined, or "categories", possibly with inheritance links, and each original class can be equipped with a type alias refering to one of these categories.

- Combined with the overload, the categories allow to realize at compile time a kind of `if...else`.

- Traits allow to associate a category with classes (as well as other properties) from the outside, without having to modify the class. Therefore, they also allow to do it with the predefined types and the classes which are not under your control.

- Traits are written by using templates and specializations.

## Exercise

In the code below, the timing of the testers make it possible to compare the copy and the move of a `std::string`, by comparing `StringCopy` and `StringMove`. For `Accumulate`, timing is not essential since we mainly care for the computation precision. We would like that the decision to time or not, rather than based on a run-time argument, relies on a test by test compile-time mechanism.

### 1. Categories

1.  Create two empty classes `TestTag` and` TestChronoTag`, and define a nested type `category` in `StringCopy` and `StringMove`, which will be an alias of `TestChronoTag`, and a nested type `category` in `Accumulate`, which will be an alias of `TestTag`.
2.  Modify the `execute_single_test()` code so that it does or not the timing, depending on whether the current `Test` is of an alias of `TestChronoTag` or `TestTag`.

### 2. Traits

Imagine that the `Accumulate` class is not under your control, and that you cannot insert the `category` definition by yourself, as you did previously. Instead, create a trait class that has the default implementation of the nested tester category, which will work fine with `StringCopy` and `StringMove`, and specialize it for `Accumulate`.

In [71]:
%%file tmp.categories-and-traits.cpp

#include <iostream>
#include <chrono>
#include <type_traits>
#include <string>

// tests

struct StringCopy
 {
  char const * const title
   {"Timing a std::string copy"} ;
  void operator()() const
   {
    std::string s1 {"dummy"}, s2 ;
    for ( int i = 0 ; i<1000 ; ++i )
    {
     if (i%2) s2 = s1 ;
     else s1 = s2 ;
    }
   }
 } ;

struct StringMove
 {
  char const * const title
   {"Timing a std::string move"} ;
  void operator()() const
   {
    std::string s1 {"dummy"}, s2 ;
    for ( int i = 0 ; i<1000 ; ++i )
     {
     if (i%2) s2 = std::move(s1) ;
     else s1 = std::move(s2) ;
     }
   }
 } ;
    
struct Accumulate
 {
  char const * const title
   {"Test accumulating 0.1"} ;
  void operator()() const
   {
    std::cout.precision(18) ;
    double acc {} ;
    for ( int i = 0 ; i<10 ; ++i )
     {
      acc += 0.1 ;
      std::cout<<acc<<std::endl ;
    }
   }
 } ;
 
// execution generic framework

template< typename TestT >
void execute_single_test( bool chrono )
 {
  TestT test ;

  std::cout<<std::endl ;
  std::cout<<"== " ;
  std::cout<<(test.title)<<std::endl ;

  if (chrono)
   {
    using namespace std::chrono ;
    auto t1 = steady_clock::now() ;
   
    test() ;
     
    auto t2 = steady_clock::now() ;
    std::cout<<"Execution time: "
      <<duration_cast<microseconds>(t2-t1).count()
      <<" us."<<std::endl ;
   }
  else test() ;
 }

template< typename... TestTs >
void execute( bool chrono )
 { ( execute_single_test<TestTs>(chrono) , ... ) ; }

// main program

int main()
 {
  execute<StringCopy,StringMove>(true) ;
  execute<Accumulate>(false) ;
 }


Overwriting tmp.categories-and-traits.cpp


In [72]:
! ( g++ -std=c++17 -Wall -pedantic-errors tmp.categories-and-traits.cpp && ./a.out )


== Timing a std::string copy
Execution time: 18 us.

== Timing a std::string move
Execution time: 11 us.

== Test accumulating 0.1
0.100000000000000006
0.200000000000000011
0.300000000000000044
0.400000000000000022
0.5
0.599999999999999978
0.699999999999999956
0.799999999999999933
0.899999999999999911
0.999999999999999889


## References

* [Effective Modern C++](http://www.amazon.fr/Effective-Modern-C-Scott-Meyers/dp/1491903996/ref=sr_1_2?ie=UTF8&qid=1410853138&sr=8-2&keywords=effective+C%2B%2B)

© *CNRS 2021*  
*Assembled and written in french by David Chamont, translated by Karim Hasnaoui, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)